In [3]:
import csv
from gmplot import gmplot
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut


In [23]:
gmap = gmplot.GoogleMapPlotter.from_geocode("San Francisco")

IndexError: list index out of range

In [25]:
gmap = gmplot.GoogleMapPlotter(40.7128, 74.0060, 13)
gmap.draw("test.html")

In [4]:
def represents_int(s):
    try: 
        int(s)
        return True
    except ValueError:
        return False

In [5]:
# Goal:
# 262 EAST 55 STREET, BROOKLYN NY 11203 =>
# 262 EAST 55th STREET, BROOKLYN NY 11203

ENDINGS = {
    '0': 'th',
    '1': 'st',
    '2': 'nd',
    '3': 'rd',
    '4': 'th',
    '5': 'th',
    '6': 'th',
    '7': 'th',
    '6': 'th',
    '8': 'th',
    '9': 'th'
}
def convert_street(addr):
    # e.g. 
    (street, rest) = addr.split(',', 1)
    str_tokens = street.split(' ')
    for i in range(len(str_tokens)):
        if i == 0: continue
        t = str_tokens[i]
        if represents_int(t):
            str_tokens[i] += ENDINGS[t[-1]]
    new_street = ' '.join(str_tokens)
    return new_street + ',' + rest

In [6]:
convert_street('483 EAST 180 STREET, BRONX NY 10457')

'483 EAST 180th STREET, BRONX NY 10457'

In [7]:
gl = Nominatim(user_agent='hri_4', timeout=10)

In [14]:
count = 0
start_counting = False
points = []
to_latlong = {}
with open('to_lat_long_4.csv', 'w') as out:
    writer = csv.DictWriter(out, fieldnames=[
        'id','geo_addr','count','lat','long'
    ])
    with open('lead_paint_processed.csv', 'r') as f:
        reader = csv.DictReader(f)
        for row in reader:
            count += 1
            if not start_counting:
                start_counting = (int(row['building_id']) == 43479)
                continue
            
            addr = convert_street(row['address'])
            try:
                loc = gl.geocode(addr)
            except GeocoderTimedOut as e:
                print('Geo Time Out: ' + addr)
                continue
            if not loc:
                print('Geo error: ' + row['building_id'] + ' | ' + addr)
                continue
            
            latlong = (loc.latitude, loc.longitude)
            writer.writerow({'id': row['building_id'],
                             'geo_addr': addr,
                             'count': row['num_violations'],
                             'lat': latlong[0],
                             'long': latlong[1]})
            
            for i in range(int(row['num_violations'])):
                points.append(latlong)

            if count % 100 == 0:
                print(str(count) + ' processed.')

2900 processed.
Geo error: 3382 | 1858 ADAM C POWELL BOULEVARD, MANHATTAN NY 10026
Geo Time Out: 610 ASHFORD STREET, BROOKLYN NY 11207
Geo error: 111437 | 830 REV JAMES POLITE AVENUE, BRONX NY 10459
3000 processed.
Geo error: 116930 | 2290 DR M L KING JR BOULEVARD, BRONX NY 10468
3100 processed.
3200 processed.
Geo error: 116917 | 2230 DR M L KING JR BOULEVARD, BRONX NY 10453
Geo error: 243222 | 960 EAST 12nd STREET, BROOKLYN NY 11230
Geo error: 811049 | 67-35A 186th LANE, QUEENS NY 11365
Geo Time Out: 300 VERNON AVENUE, BROOKLYN NY 11206
Geo error: 213152 | 3000 BRIGHTON 12nd STREET, BROOKLYN NY 11235
3300 processed.
Geo Time Out: 2310 CRESTON AVENUE, BRONX NY 10468
Geo error: 209768 | 2105 BEVERLY ROAD, BROOKLYN NY 11226
Geo error: 335368 | 138 MC KINLEY AVENUE, BROOKLYN NY 11208
Geo error: 287660 | 1566 EASTERN PARKWAY EXTENSION, BROOKLYN NY 11233
3400 processed.
Geo error: 213148 | 2954 BRIGHTON 12nd STREET, BROOKLYN NY 11235
Geo error: 95436 | 215 MC CLELLAN STREET, BRONX NY 10456

Geo error: 93899 | 4568 MANH COLLEGE PARKWAY, BRONX NY 10471
Geo error: 95437 | 225 MC CLELLAN STREET, BRONX NY 10456
Geo error: 3419 | 1990 ADAM C POWELL BOULEVARD, MANHATTAN NY 10026
6900 processed.
Geo error: 116944 | 2332 DR M L KING JR BOULEVARD, BRONX NY 10468
Geo error: 242591 | 1629 EAST 12nd STREET, BROOKLYN NY 11229
Geo error: 67857 | 801 EAST 213rd STREET, BRONX NY 10467
Geo error: 11966 | 444 EAST 13rd STREET, MANHATTAN NY 10009
7000 processed.
Geo error: 137940 | 270 12nd STREET, BROOKLYN NY 11215
Geo Time Out: 97-25 64th AVENUE, QUEENS NY 11374
Geo error: 84040 | 2065 HAVILAND AVENUE, BRONX NY 10472
Geo error: 375404 | 68 ST PAULS PLACE, BROOKLYN NY 11226
7100 processed.
Geo error: 523478 | 39-02 111st STREET, QUEENS NY 11368
Geo error: 243459 | 1580 EAST 13rd STREET, BROOKLYN NY 11230
Geo error: 116818 | 1633 DR M L KING JR BOULEVARD, BRONX NY 10453
Geo error: 868877 | 1183 EAST 213rd STREET, BRONX NY 10469
7200 processed.
Geo error: 116934 | 2300 DR M L KING JR BOULEVAR

In [20]:
gl.geocode("1055 Dr Martin L King Jr Blvd, Bronx, NY 10452")

In [21]:
gl.geocode("280 PARK HILL AVENUE, STATEN ISLAND NY 10304")

Location(280, Park Hill Avenue, Clifton, Todt Hill, Staten Island, Richmond County, NYC, New York, 10304, USA, (40.61203475, -74.0830785191273, 0.0))

In [26]:
# Place map
gmap = gmplot.GoogleMapPlotter(37.766956, -122.438481, 13)

# Polygon
golden_gate_park_lats, golden_gate_park_lons = zip(*[
    (37.771269, -122.511015),
    (37.773495, -122.464830),
    (37.774797, -122.454538),
    (37.771988, -122.454018),
    (37.773646, -122.440979),
    (37.772742, -122.440797),
    (37.771096, -122.453889),
    (37.768669, -122.453518),
    (37.766227, -122.460213),
    (37.764028, -122.510347),
    (37.771269, -122.511015)
    ])
gmap.plot(golden_gate_park_lats, golden_gate_park_lons, 'cornflowerblue', edge_width=10)

# Scatter points
top_attraction_lats, top_attraction_lons = zip(*[
    (37.769901, -122.498331),
    (37.768645, -122.475328),
    (37.771478, -122.468677),
    (37.769867, -122.466102),
    (37.767187, -122.467496),
    (37.770104, -122.470436)
    ])
gmap.scatter(top_attraction_lats, top_attraction_lons, '#3B0B39', size=40, marker=False)

# Marker
hidden_gem_lat, hidden_gem_lon = 37.770776, -122.461689
gmap.marker(hidden_gem_lat, hidden_gem_lon, 'cornflowerblue')

# Draw
gmap.draw("my_map.html")

In [9]:
import pandas as pd
import folium
import random as rand

In [5]:
df = pd.read_csv("lat_long_all.csv")
df

,building_id,address,num_violations,latitude,longitude
0,64592,"361 EAST 163rd STREET, BRONX NY 10451",106,40.826502,-73.914936
1,73440,"710 EAST 243rd STREET, BRONX NY 10470",92,40.906259,-73.848707
2,65811,"250 EAST 176th STREET, BRONX NY 10457",76,40.847325,-73.904915
3,121007,"80 WEST 170th STREET, BRONX NY 10452",70,40.840239,-73.922234
4,355498,"1690 PRESIDENT STREET, BROOKLYN NY 11213",67,40.666832,-73.930961
5,217336,"1119 CARROLL STREET, BROOKLYN NY 11225",63,40.667526,-73.952467
6,217335,"1116 CARROLL STREET, BROOKLYN NY 11225",63,40.667239,-73.953838
7,109434,"1221 SHERIDAN AVENUE, BRONX NY 10456",62,40.834645,-73.916352
8,122197,"1206 WESTCHESTER AVENUE, BRONX NY 10459",62,40.825869,-73.889380
9,355164,"1180 PRESIDENT STREET, BROOKLYN NY 11225",61,40.667872,-73.951458


In [13]:
coords = list(zip(df['latitude'], df['longitude'], df['num_violations']))
coords[:5]

[(40.8265016363636, -73.9149357272727, 106),
 (40.9062594693878, -73.8487072244898, 92),
 (40.847325, -73.9049147, 76),
 (40.84023870000001, -73.92223409999998, 70),
 (40.6668317, -73.9309615, 67)]

In [14]:
coords[0][0:2]

(40.8265016363636, -73.9149357272727)

In [15]:
#New York City Coordinates
NYC_COORD = [40.7128, -74.0059]

# Sample (0.33% over 1.5 million) 
sample_coords = rand.sample(list(coords),500)

# Build map 
map_nyc = folium.Map(location=NYC_COORD, zoom_start=12, 
tiles='cartodbpositron', width=640, height=480)

def get_color(count):
    if count > 40:
        return '#ff0000'
    elif count > 20:
        return '#ff4444'
    elif count > 10:
        return '#ff8888'
    elif count > 5:
        return '#ffaaaa'
    else:
        return '#ffcccc'

# Plot coordinates using comprehension list
[folium.CircleMarker(sample_coords[i][0:2], radius=0.1,
                color=get_color(sample_coords[i][2]), 
                fill_color=get_color(sample_coords[i][2])).add_to(map_nyc) 
for i in range(len(sample_coords))]

# Display map in Jupyter
map_nyc